### Dependencies and setup

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import config
import json as json
from config import gmaps_key

In [2]:
cities_vac_df = pd.read_csv('../output_data/cities.csv')
cities_vac_df.dropna(inplace = True) 
cities_vac_df

,Unnamed: 0,City,Lat,Lon,Temp_C,Feels_like,Temp_min_C,Temp_max_C,Humidity,Wind_speed_mph,Clouds,Country,Time_zone
0,1,beloha,-25.17,45.05,15.51,12.19,15.51,15.51,74,5.16,0,MG,10800
1,2,albany,42.60,-73.97,29.69,30.45,27.22,31.67,42,1.43,65,US,-14400
2,3,diego de almagro,-26.37,-70.05,17.80,12.12,17.80,17.80,18,4.12,0,CL,-14400
3,4,mataura,-46.19,168.86,6.75,3.48,6.75,6.75,83,2.80,94,NZ,43200
4,5,souillac,-20.52,57.52,20.00,15.27,20.00,20.00,56,7.20,75,MU,14400
...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,623,sao jose da coroa grande,-8.90,-35.15,23.75,23.61,23.75,23.75,76,4.99,1,BR,-10800
578,624,mezen,65.85,44.24,14.74,14.86,14.74,14.74,93,1.45,34,RU,10800
579,625,sedlcany,49.66,14.43,18.43,18.35,17.78,18.89,70,1.38,73,CZ,7200
580,626,nur,52.67,22.32,15.53,13.05,15.53,15.53,82,4.63,18,PL,7200


### Humidity Heatmap

In [3]:
gmaps.configure(api_key=gmaps_key)
locations = cities_vac_df[["Lat", "Lon"]].astype(float)
humidity = cities_vac_df["Humidity"].astype(float)

fig = gmaps.figure()

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)

# Shoe map
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Cities Selected by Weather

In [4]:
#This is kind of bogus because data are for one timepoint only... Could be a sunny today day in Bergen, 
#it happens, but you might be disappointed on arrival.

#Drop any rows will null values - already done above.

cities_pref_df = cities_vac_df
cities_pref_df.drop(cities_pref_df[cities_pref_df['Feels_like'] > 30].index, inplace = True)
cities_pref_df.drop(cities_pref_df[cities_pref_df['Feels_like'] < -10].index, inplace = True)
cities_pref_df.drop(cities_pref_df[cities_pref_df['Humidity'] > 80].index, inplace = True)
cities_pref_df.drop(cities_pref_df[cities_pref_df['Country'] == "US"].index, inplace = True)
cities_pref_df



,Unnamed: 0,City,Lat,Lon,Temp_C,Feels_like,Temp_min_C,Temp_max_C,Humidity,Wind_speed_mph,Clouds,Country,Time_zone
0,1,beloha,-25.17,45.05,15.51,12.19,15.51,15.51,74,5.16,0,MG,10800
2,3,diego de almagro,-26.37,-70.05,17.80,12.12,17.80,17.80,18,4.12,0,CL,-14400
4,5,souillac,-20.52,57.52,20.00,15.27,20.00,20.00,56,7.20,75,MU,14400
7,10,belmonte,-15.86,-38.88,23.00,23.10,23.00,23.00,64,2.60,20,BR,-10800
8,11,upernavik,72.79,-56.15,8.12,5.84,8.12,8.12,78,1.51,44,GL,-7200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,618,tereze,43.94,42.44,27.00,24.18,27.00,27.00,28,3.00,55,RU,10800
574,620,zabol,32.25,67.25,20.50,19.16,20.50,20.50,63,3.35,0,AF,16200
577,623,sao jose da coroa grande,-8.90,-35.15,23.75,23.61,23.75,23.75,76,4.99,1,BR,-10800
579,625,sedlcany,49.66,14.43,18.43,18.35,17.78,18.89,70,1.38,73,CZ,7200


### Hotel Search

In [5]:
hotel_df = cities_pref_df
hotel_df["Hotel"] = ""

target_radius = 5000
target_type = "lodging"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {
    "radius": target_radius,
    "type": target_type,
    "key": gmaps_key
}

#(print log commented out because annoying)

for index, row in hotel_df.iterrows():
    # get lat, lon from df; remaining parameters from parameters dictionary
    lat = row["Lat"]
    lon = row["Lon"]
    params["location"] = f"{lat},{lon}"

    response = requests.get(base_url, params=params).json()
    results = response['results']    
    #print(json.dumps(hotel_name, indent=4, sort_keys=True))
    try:
        #print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel"] = results[0]['name']
        
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel"] = "missing value"
    #print("------------")

hotel_df

,Unnamed: 0,City,Lat,Lon,Temp_C,Feels_like,Temp_min_C,Temp_max_C,Humidity,Wind_speed_mph,Clouds,Country,Time_zone,Hotel
0,1,beloha,-25.17,45.05,15.51,12.19,15.51,15.51,74,5.16,0,MG,10800,missing value
2,3,diego de almagro,-26.37,-70.05,17.80,12.12,17.80,17.80,18,4.12,0,CL,-14400,hotel luna del desierto
4,5,souillac,-20.52,57.52,20.00,15.27,20.00,20.00,56,7.20,75,MU,14400,Bils' Villa
7,10,belmonte,-15.86,-38.88,23.00,23.10,23.00,23.00,64,2.60,20,BR,-10800,Pousada Monte Carmelo O Pão
8,11,upernavik,72.79,-56.15,8.12,5.84,8.12,8.12,78,1.51,44,GL,-7200,Gina's Guesthouse - Upernavik
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,618,tereze,43.94,42.44,27.00,24.18,27.00,27.00,28,3.00,55,RU,10800,missing value
574,620,zabol,32.25,67.25,20.50,19.16,20.50,20.50,63,3.35,0,AF,16200,missing value
577,623,sao jose da coroa grande,-8.90,-35.15,23.75,23.61,23.75,23.75,76,4.99,1,BR,-10800,Crôa Mares Hotel
579,625,sedlcany,49.66,14.43,18.43,18.35,17.78,18.89,70,1.38,73,CZ,7200,Junácká základna Červený Hrádek


### Hotel Map

In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [7]:
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [8]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…